# ComCam pointing errors

Craig Lage - 14-Nov-24

In [ ]:
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay

In [ ]:
butler = butlerUtils.makeDefaultButler("LSSTComCam")
instrument = 'LSSTComCam'

startDay = 20241028
endDay = 20241119

els = []
azs = []
ras = []
decs = []
deltaRas = []
deltaDecs = []

filters = []
pressures = []
temps = []
hums = []
times = []
dayObs = startDay
while dayObs <= endDay:
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", 
                where=f"exposure.day_obs={dayObs} and instrument='LSSTComCam'"):
        exposureList.append([record.id, record])
    exposureList.sort(key=lambda x: x[0])
    #print(len(exposureList))
    for [id,record] in exposureList:
        if record.observation_type not in ['acq', 'science', 'focus']:
            continue
        try:
            calExp = butler.get('calexp', detector=4, visit=record.id, instrument=instrument)
            rawExp = butler.get('raw', detector=4, exposure=record.id, instrument=instrument)
            md = rawExp.getMetadata()
            els.append(md['ELSTART'])
            azs.append(md['AZSTART'])
            filters.append(md['FILTBAND'])
            pressures.append(md['PRESSURE'])
            temps.append(md['AIRTEMP'])
            hums.append(md['HUMIDITY'])
            times.append((md['MJD-BEG'] + md['MJD-END']) / 2.0)
            cWcs = calExp.getWcs()
            rWcs = rawExp.getWcs()
            rawSkyCenter = rWcs.getSkyOrigin()
            calExpSkyCenter = cWcs.pixelToSky(rWcs.getPixelOrigin())
            ra = calExpSkyCenter.getRa().asDegrees()
            dec = calExpSkyCenter.getDec().asDegrees()
            ras.append(ra)
            decs.append(dec)
            #print(ra, dec)
            deltaRa = rawSkyCenter.getRa().asArcseconds() - calExpSkyCenter.getRa().asArcseconds()
            deltaDec = rawSkyCenter.getDec().asArcseconds() - calExpSkyCenter.getDec().asArcseconds()
            deltaRas.append(deltaRa)
            deltaDecs.append(deltaDec)
            #print(deltaRa, deltaDec)
        except:
            continue
    print(dayObs, len(ras))
    dayObs = calcNextDay(dayObs)



In [ ]:
filename = "/home/c/cslage/u/ComCam/data/pointing_results_19nov24.pkl"
with open(filename, 'wb') as f:
    pkl.dump([els, azs, ras, decs, deltaRas, deltaDecs, pressures, temps, hums, times, filters], f)


In [ ]:
filename = "/home/c/cslage/u/ComCam/data/pointing_results_19nov24.pkl"
with open(filename, 'rb') as f:
    [els, azs, ras, decs, deltaRas, deltaDecs, pressures, temps, hums, times, filters] = pkl.load(f)
len(els)

In [ ]:
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION
wavelengths = {'u':3671, 'g':4827, 'r':6223, 'i':7546, 'z':8691, 'y':9712}

In [ ]:
deltaAzs = []
deltaEls = []
bad_indices = []
for index in range(len(ras)):
    skyLocation = SkyCoord(ras[index]*u.deg, decs[index]*u.deg)
    time = Time(times[index], format='mjd', scale='tai')
    pressure = pressures[index] * u.pascal
    temperature = temps[index] * u.Celsius
    hum = hums[index]
    wl = wavelengths[filters[index]] * u.angstrom
    altAz = AltAz(obstime=time, location=SIMONYI_LOCATION, pressure=pressure, 
                 temperature=temperature, relative_humidity=hum, obswl=wl)
    obsAltAz = skyLocation.transform_to(altAz)
    deltaAz = azs[index] - obsAltAz.az.deg
    if deltaAz > 360.0:
        deltaAz -= 360.0
    if deltaAz < -180.0:
        deltaAz += 360.0
    deltaAz *= 3600.0 * np.cos(obsAltAz.alt.rad)
    deltaEl = (els[index] - obsAltAz.alt.deg) * 3600.0
    deltaAzs.append(deltaAz)
    deltaEls.append(deltaEl)
    """
    if abs(deltaEl) < 450.0:
        deltaAzs.append(deltaAz)
        deltaEls.append(deltaEl)
    else:
        print(index)
        bad_indices.append(index)
        
for index in bad_indices:
    del azs[index]
    del els[index]
"""
    
len(els)        

In [ ]:
plt.figure(figsize=(8,8))
plt.suptitle("ComCam Delta RA/Dec 2024-10-24 - 2024-11-19")
plt.subplots_adjust(hspace=0.3, wspace=0.3)
plt.subplot(2,2,1)
plt.scatter(decs, deltaDecs)
plt.xlabel('Dec (deg)')
plt.xlim(-90, 0)
plt.ylabel('Delta Dec arcsec')
plt.subplot(2,2,2)
plt.scatter(ras, deltaDecs)
plt.xlabel('Ra (deg)')
plt.xlim(0,360)
plt.ylabel('Delta Dec arcsec')
plt.subplot(2,2,3)
plt.scatter(decs, deltaRas)
plt.xlabel('Dec (deg)')
plt.xlim(-90, 0)
plt.ylabel('Delta Ra arcsec')
plt.subplot(2,2,4)
plt.scatter(ras, deltaRas)
plt.xlabel('Ra (deg)')
plt.xlim(0,360)
plt.ylabel('Delta Ra arcsec')
plt.savefig("/home/c/cslage/u/ComCam/data/Delta_RaDec_20Nov24.png")


In [ ]:
from scipy.optimize import minimize
def FOM(params, args):
    fom = 0.0
    [azs, deltaEls] = args
    [amp, off] = params
    for i in range(len(azs)):
        if abs(deltaEls[i]) > 450.0:
            continue
        model = off + amp * np.sin((azs[i]) * np.pi / 180.0)
        fom += np.square(model - deltaEls[i])
    return fom

args = [azs, deltaEls]
x0 = [400.0, 0.0]
result = minimize(FOM, x0, args=args, method='Powell')
result

In [ ]:
from scipy.optimize import minimize
def FOM(params, args):
    fom = 0.0
    [azs, deltaAzs] = args
    [amp, off] = params
    for i in range(len(azs)):
        if abs(deltaAzs[i]) > 450.0:
            continue
        model = off + amp * np.cos(0.5 * (azs[i]) * np.pi / 180.0)
        fom += np.square(model - deltaAzs[i])
    return fom

args2 = [azs, deltaAzs]
x0 = [400.0, 0.0]
result2 = minimize(FOM, x0, args=args2, method='Powell')
result2

In [ ]:
[amp, off] = result.x
[amp2, off2] = result2.x
xs = np.linspace(-180.0, 180.0, 200)
ys = off + amp * np.sin((xs) * np.pi / 180.0)
ys2 = off2 + amp2 * np.cos(0.5 * (xs) * np.pi / 180.0)

plt.figure(figsize=(8,8))
plt.suptitle("ComCam Delta AltAz 2024-10-24 - 2024-11-19")
plt.subplots_adjust(hspace=0.3, wspace=0.7)
plt.subplot(2,2,1)
p1 = plt.scatter(els, deltaEls, c=azs, cmap=plt.cm.coolwarm)
cb1 = plt.colorbar(p1)
cb1.set_label('Az')
plt.xlabel('El')
plt.xlim(0,90)
plt.ylabel('Delta El arcsec')
plt.subplot(2,2,2)
p2 = plt.scatter(azs, deltaEls,c=els, cmap=plt.cm.coolwarm)
cb2 = plt.colorbar(p2)
cb2.set_label('El')
plt.plot(xs, ys, ls = '--', color='black')
plt.text(-100,700,f"deltaEl={off:.1f}+\n{amp:.1f}*sin(az)")
plt.xlabel('Az')
plt.xlim(-180, 180)
plt.ylabel('Delta El arcsec')
plt.subplot(2,2,3)
p3 = plt.scatter(els, deltaAzs, c=azs, cmap=plt.cm.coolwarm)
cb3 = plt.colorbar(p3)
cb3.set_label('Az')
plt.xlabel('El')
plt.xlim(0,90)
plt.ylabel('Delta Az*cos(El) arcsec')
plt.subplot(2,2,4)
p4 = plt.scatter(azs, deltaAzs,c=els, cmap=plt.cm.coolwarm)
plt.plot(xs, ys2, ls = '--', color='black')
plt.text(-100,-400,f"deltaAz={off2:.1f}+\n{amp2:.1f}*cos(az/2)")
cb4 = plt.colorbar(p4)
cb4.set_label('El')
plt.xlabel('Az')
plt.xlim(-180, 180)
plt.ylabel('Delta Az*cos(El) arcsec')
plt.savefig("/home/c/cslage/u/ComCam/data/Delta_AltAz_20Nov24.png")


# Generate a dummy tpoint input file with these errors

In [ ]:
outfilename = "/home/c/cslage/u/ComCam/data/Dummy_Tpoint_Input.dat"
outfile = open(outfilename, 'w')
outfile.write("!" + outfilename + "\n")
outfile.write("!Simonyi Telescope Dummy file, 2024 Nov 20 24 \n")
outfile.write(": ALTAZ\n")
outfile.write(": ROTNR\n")
outfile.write("-30 14 40.2\n")
rot = 0.0
for az in range(-180, 180, 15):
    for el in range(30, 90, 10):
        deltaEl = (off + amp * np.sin((az) * np.pi / 180.0))
        deltaAz = (off2 + amp2 * np.cos(0.5 * (az) * np.pi / 180.0)) / np.cos(el * np.pi / 180.0)
        elReal = el + deltaEl / 3600.0
        azReal = az + deltaAz / 3600.0
        outfile.write(f"{azReal:.6f}\t{elReal:.6f}\t{az:.6f}\t{el:.6f}\t{rot:.6f}\n")
outfile.write("END\n")
outfile.close()



## Test if that's how the AzEl coordinates respond to a rotation about one of the axes

In [ ]:
def to_xyz(az, el):
    x = np.cos(el)*np.cos(az)
    y = np.cos(el)*np.sin(az)
    z = np.sin(el)
    return (x,y,z)

def to_azel(x, y, z):
    el = np.arcsin(z)
    az = np.arctan2(y, x)    
    return az, el
    
def Ry(theta):
  return np.matrix([[ np.cos(theta), 0, np.sin(theta)],
                   [ 0           , 1, 0           ],
                   [-np.sin(theta), 0, np.cos(theta)]])

def Rx(theta):
  return np.matrix([[ 1, 0           , 0           ],
                   [ 0, np.cos(theta),-np.sin(theta)],
                   [ 0, np.sin(theta), np.cos(theta)]])


In [ ]:
# Check if we recover the initial AzEl
az = 153.0; el = 65.3
(x,y,z) = to_xyz(az*np.pi/180.0, el*np.pi/180.0)
print((x,y,z))
az2, el2 = to_azel(x,y,z)
print(az2*180.0/np.pi, el2*180.0/np.pi)

In [ ]:
test_azs = []
test_els = []
rot_azs = []
rot_els = []
test_deltaAzs = []
test_deltaEls = []
rot = 200.0 / 3600.0 * np.pi / 180.0
rx = Rx(rot)
for az in range(-175, 185, 15):
    for el in range(40, 90, 10):
        test_azs.append(az)
        test_els.append(el)
        (x,y,z) = to_xyz(az*np.pi/180.0, el*np.pi/180.0)
        out = rx.dot((x,y,z))
        out = np.array(out)[0]
        az2, el2 = to_azel(out[0], out[1], out[2])
        az2 = az2*180.0/np.pi
        el2 = el2*180.0/np.pi
        rot_azs.append(az2)
        rot_els.append(el2)
        test_deltaAz = (az - az2) * 3600.0
        test_deltaEl = (el - el2) * 3600.0
        test_deltaAzs.append(test_deltaAz * np.cos(el*np.pi/180.0))
        test_deltaEls.append(test_deltaEl)





In [ ]:
plt.figure(figsize=(8,8))
plt.suptitle("Test DeltaAz")
plt.subplots_adjust(hspace=0.3, wspace=0.7)
plt.subplot(2,2,1)
p1 = plt.scatter(test_els, test_deltaEls, c=test_azs, cmap=plt.cm.coolwarm)
cb1 = plt.colorbar(p1)
cb1.set_label('Az')
plt.xlabel('El')
plt.xlim(0,90)
plt.ylabel('Delta El arcsec')
plt.subplot(2,2,2)
p2 = plt.scatter(test_azs, test_deltaEls,c=test_els, cmap=plt.cm.coolwarm)
cb2 = plt.colorbar(p2)
cb2.set_label('El')
plt.xlabel('Az')
plt.xlim(-180, 180)
plt.ylabel('Delta El arcsec')
plt.subplot(2,2,3)
p3 = plt.scatter(test_els, test_deltaAzs, c=test_azs, cmap=plt.cm.coolwarm)
cb3 = plt.colorbar(p3)
cb3.set_label('Az')
plt.xlabel('El')
plt.xlim(0,90)
plt.ylabel('Delta Az*cos(El) arcsec')
plt.subplot(2,2,4)
p4 = plt.scatter(test_azs, test_deltaAzs,c=test_els, cmap=plt.cm.coolwarm)
cb4 = plt.colorbar(p4)
cb4.set_label('El')
plt.xlabel('Az')
plt.xlim(-180, 180)
plt.ylabel('Delta Az*cos(El) arcsec')
plt.savefig("/home/c/cslage/u/ComCam/data/Test_Delta_AltAz_18Nov24.png")


In [ ]:
rad_azs = np.array(azs) * np.pi / 180.0

fig=plt.figure(figsize=(8,8))
ax1 = plt.subplot(111, projection='polar')
ax1.set_title(f"Sky coverage - ComCam campaign 20241028-20241119")
ax1.invert_yaxis()
ax1.set_rlim(90,0)
ax1.scatter (rad_azs, els)
plt.savefig("/home/c/cslage/u/ComCam/data/Sky_Coverage_20Nov24.png")

In [ ]:
butler = Butler('/sdf/group/rubin/repo/main', collections=["LSSTComCam/raw/all","LSSTComCam/calib"])
instrument = 'LSSTComCam'

def DeltaAltAz (el, az):
    # This calculates the offset due to the pointing model
    # as extracted from ComCam images.
    deltaEl = 125.4 - 187.9 * np.sin(az * np.pi / 180.0)
    deltaAz = 222.0 + 348.1 * np.cos(az * np.pi / 360.0)
    elPrime = el + deltaEl / 3600.0
    azPrime = az + deltaAz / 3600.0
    return [elPrime, azPrime]

def CalculateDrift(expId):
    rawExp = butler.get('raw', detector=4, exposure=expId, instrument=instrument)
    md = rawExp.getMetadata()
    filter = md['FILTBAND']
    pressure = md['PRESSURE'] * u.pascal
    temperature = md['AIRTEMP'] * u.Celsius
    hum = md['HUMIDITY']
    time1 = Time(md['MJD-BEG'], format='mjd', scale='tai')
    time2 = Time(md['MJD-END'], format='mjd', scale='tai')
    ra = md['RASTART']
    dec = md['DECSTART']
    skyLocation = SkyCoord(ra*u.deg, dec*u.deg)
    wl = wavelengths[filter] * u.angstrom
    altAz1 = AltAz(obstime=time1, location=SIMONYI_LOCATION, pressure=pressure, 
                 temperature=temperature, relative_humidity=hum, obswl=wl)
    altAz2 = AltAz(obstime=time2, location=SIMONYI_LOCATION, pressure=pressure, 
                 temperature=temperature, relative_humidity=hum, obswl=wl)
    obsAltAz1 = skyLocation.transform_to(altAz1)
    obsAltAz2 = skyLocation.transform_to(altAz2)
    # 1 is at the beginning of the exposure, 2 is at the end
    # el, az are the actual values, prime values reflect the pointing model
    # These are all in degrees
    el1 = obsAltAz1.alt.deg
    az1 = obsAltAz1.az.deg
    el2 = obsAltAz2.alt.deg
    az2 = obsAltAz2.az.deg
    [elPrime1, azPrime1] = DeltaAltAz (el1, az1)
    [elPrime2, azPrime2] = DeltaAltAz (el2, az2)
    # Change values are the change from the beginning to the end of the exposure, in arcseconds
    azChange = (az2 - az1) * 3600.0
    elChange = (el2 - el1) * 3600.0
    azPrimeChange = (azPrime2 - azPrime1) * 3600.0
    elPrimeChange = (elPrime2 - elPrime1) * 3600.0
    azDrift = azChange - azPrimeChange
    elDrift = elChange - elPrimeChange
    print(f" For {expId}, Azimuth drift = {azDrift:.2f} arcseconds, Elevation drift = {elDrift:.2f} arcseconds")
    return

CalculateDrift(2024120700551)